## Импорт модели Word2Vec

In [1]:
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors

In [2]:
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [4]:
wv.wv.most_similar('😠')

[('виктория', 0.9023991227149963),
 ('бегать', 0.9014540314674377),
 ('😰', 0.9013167023658752),
 ('нынешний', 0.901007354259491),
 ('вооружение', 0.900209903717041),
 ('медицина', 0.8998721837997437),
 ('приходить', 0.8997824192047119),
 ('приводить', 0.8997141122817993),
 ('плевать', 0.8997119665145874),
 ('тихий', 0.89943528175354)]